In [2]:
import os
import json
import jsonlines

In [3]:
result_dir = "../../../results"
input_dir = "../../../preprocessed_datasets"

In [8]:
model_name = "Llama-3.1-8B-Instruct"
dataset_name = "CommonsenseQA"
prompting_strategy = "zero-shot"

output_dir = f"{result_dir}/{dataset_name}/{model_name}"

predictions_path = os.path.join(output_dir, f"{prompting_strategy}_activation_steering_5.0_predictions.jsonl")
base_predictions_path = os.path.join(output_dir, f"{prompting_strategy}_predictions.jsonl")

In [9]:
with jsonlines.open(predictions_path) as fin:
    consistency_list = []
    for example in fin.iter():
        set_consistency = int(example["consistency"]["mean"] > 0.99)
        consistency_list.append(set_consistency)

with jsonlines.open(base_predictions_path) as fin:
    base_consistency_list = []
    for example in fin.iter():
        set_consistency = int(example["consistency"]["mean"] > 0.99)
        base_consistency_list.append(set_consistency)

In [15]:
good, bad, neutral, total = 0, 0, 0, 0
original_error = 0

for score, base_score in zip(consistency_list, base_consistency_list):
    if score > base_score:
        good += 1
    elif score < base_score:
        bad += 1
    else:
        neutral += 1
    if base_score < 0.5:
        original_error += 1
    total += 1

In [18]:
print(f"Total: {total} samples")
print("="*20)
print(f"Inconsistency error (baseline): {original_error} samples")
print()
print(f"0 => 1 (improvement): {good} samples")
print(f"1 => 0 (side effect): {bad} samples")
print(f"Net improvement: {good - bad} samples")
print()
print(f"Unchanged: {neutral} samples")

Total: 977 samples
Inconsistency error (baseline): 96 samples

0 => 1 (improvement): 7 samples
1 => 0 (side effect): 5 samples
Net improvement: 2 samples

Unchanged: 965 samples
